## An Automated Portfolio Trading System with Feature Preprocessing and Recurrent Reinforcement Learning

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.data import Data, get_buy_and_hold_returns, INDICATORS

from lib.model import RRL, train, validation

from lib.backtest import (
    run_rrl_strategy, 
    create_backtest_dataset, 
    generate_layout, 
    generate_nrows, 
    generate_subplot_titles, 
    init_subplots, 
    populate_subplots,
    plot_cumulative_profits, 
    generate_df_barplot, 
    make_cumrets_barplot,
)

from lib.metrics import (
    calc_cumulative_profits, 
    calc_sharpe_ratio, 
)

In [3]:
import numpy as np 
import pandas as pd

from pickle import load, dump

### Data

#### Settings

In [197]:
# assets in the portfolio
assets = ["XOM", "VZ", "NKE", "AMAT", "MCD", "MSFT", "AAP", "NOV"]

# date settings
start_date = "2009-12-31"
end_date = "2017-12-29"
window_size = 100

# number of principal components (None means no PCA)
pca_ncp = None

# technical indicators
momentum = [
    "MOM",
    "MACD",
    "MFI",
    "RSI"
]

volatility = [
    "ATR",
    "NATR"
]

cycle = [ 
    # "HTDCP",
    # "HTS",
    # "HTTMMM"
]

volume = [
    "CO",
    "OBV"
]

indicators = momentum + volatility + cycle + volume
indicators

['MOM', 'MACD', 'MFI', 'RSI', 'ATR', 'NATR', 'CO', 'OBV']

#### Instantiate `Data` object

In [198]:
data = Data(start_date, end_date, window_size, assets, indicators, pca_ncp)
data

[*********************100%***********************]  8 of 8 completed


Data(start_date='2009-12-31', end_date='2017-12-29', assets=['XOM', 'VZ', 'NKE', 'AMAT', 'MCD', 'MSFT', 'AAP', 'NOV'], indicators=['MOM', 'MACD', 'MFI', 'RSI', 'ATR', 'NATR', 'CO', 'OBV'], window_size=100)

In [138]:
buy_and_hold_returns = get_buy_and_hold_returns(data.df)

### `RRL` - Training and validation

In [199]:
initial_invest = 100
n_epochs = 50

#### No transaction fees 

$\delta=0$

In [200]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features) 
print(rrl) 

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_no_fees_5ncp.pkl"
file_path = "./backup/no_pca/rrl_no_fees_2.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:   8%|▊         | 4/50 [00:00<00:01, 28.94it/s, sharpe_ratio=0.357]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 2.240829495772644
Cumulative profits: 120.65850820091049
Training window: 2010-06-30/2010-10-14


Training in progress...:  16%|█▌        | 8/50 [00:00<00:01, 30.24it/s, sharpe_ratio=2.27]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 2.1490217436369385
Cumulative profits: 139.96064801732425
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -0.2439999298823191
Cumulative profits: 289.93379425602114
Training window: 2017-03-31/2017-07-18


Training in progress...:  16%|█▌        | 8/50 [00:00<00:01, 27.39it/s, sharpe_ratio=-.235]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 2.0695119189214943
Cumulative profits: 337.6749848185656
Training window: 2017-08-23/2017-12-07


Training in progress...:  14%|█▍        | 7/50 [00:00<00:01, 25.04it/s, sharpe_ratio=2.44]


Less than 30 observations in the validation set after preprocessing.


#### Transactions fees 

$\delta \in \{0.001, 0.003, 0.005\}$

In [201]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.001) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_10bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_10bps_2.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0.001, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  30%|███       | 15/50 [00:00<00:01, 29.54it/s, sharpe_ratio=0.261]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 2.0809343395150592
Cumulative profits: 119.47381968292655
Training window: 2010-06-30/2010-10-14


Training in progress...:  28%|██▊       | 14/50 [00:00<00:01, 28.22it/s, sharpe_ratio=1.99]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 1.6444521919131483
Cumulative profits: 132.8164845568626
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -0.9399952940140147
Cumulative profits: 184.53255936617572
Training window: 2017-03-31/2017-07-18


Training in progress...:   6%|▌         | 3/50 [00:00<00:02, 22.09it/s, sharpe_ratio=-.993]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 2.2378936313600812
Cumulative profits: 216.2132892119679
Training window: 2017-08-23/2017-12-07


Training in progress...:  20%|██        | 10/50 [00:00<00:01, 23.28it/s, sharpe_ratio=2.21]


Less than 30 observations in the validation set after preprocessing.


In [202]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.003) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_30bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_30bps_2.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0.003, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  22%|██▏       | 11/50 [00:00<00:01, 28.55it/s, sharpe_ratio=0.0433]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 1.6787823664130732
Cumulative profits: 115.53158420592541
Training window: 2010-06-30/2010-10-14


Training in progress...:  30%|███       | 15/50 [00:00<00:01, 17.64it/s, sharpe_ratio=1.43]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 1.4241667392589288
Cumulative profits: 126.99981843214958
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -1.1353962135229705
Cumulative profits: 101.53578103208287
Training window: 2017-03-31/2017-07-18


Training in progress...:  38%|███▊      | 19/50 [00:00<00:00, 33.56it/s, sharpe_ratio=-.923]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 2.1417205504316446
Cumulative profits: 114.43878096759616
Training window: 2017-08-23/2017-12-07


Training in progress...:  24%|██▍       | 12/50 [00:00<00:01, 28.79it/s, sharpe_ratio=0.673]


Less than 30 observations in the validation set after preprocessing.


In [203]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.005) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_30bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_50bps_2.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0.005, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  66%|██████▌   | 33/50 [00:01<00:00, 30.38it/s, sharpe_ratio=0.154] 


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 1.2289159857256229
Cumulative profits: 110.73468642363189
Training window: 2010-06-30/2010-10-14


Training in progress...:  14%|█▍        | 7/50 [00:00<00:01, 28.75it/s, sharpe_ratio=1.4] 


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 0.8526040420904614
Cumulative profits: 117.03211790303087
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -1.6780571053072002
Cumulative profits: 64.8903442257465
Training window: 2017-03-31/2017-07-18


Training in progress...:  44%|████▍     | 22/50 [00:00<00:00, 34.00it/s, sharpe_ratio=-2.3] 


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 1.8373559637300583
Cumulative profits: 72.29897020649139
Training window: 2017-08-23/2017-12-07


Training in progress...: 100%|██████████| 50/50 [00:01<00:00, 35.83it/s, sharpe_ratio=0.487]


Less than 30 observations in the validation set after preprocessing.


### Backtest

In [204]:
file_names = [f"rrl_{fees}_2" for fees in ("no_fees", "10bps", "30bps", "50bps")]

path = "./backup/no_pca/"
results = {
    f.replace("results_", ""): load(open(f"{path}{f}.pkl", "rb"))
    for f in file_names
}

In [205]:
dfs_backtest = {
    k: create_backtest_dataset(v)
    for k, v in results.items()
}

#### Cumulative profits

In [206]:
initial_invest = 100

In [207]:
layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title="", 
    x_axis_args={"gridcolor": "lightgrey"}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "$"}}
)
fig = plot_cumulative_profits(
    dfs_backtest, 
    buy_and_hold_returns, 
    initial_invest, 
    layout)

fig.show()

#### Analysis per trading window for one setting

In [213]:
setting = "rrl_no_fees_2"

df = dfs_backtest[setting]
batch_names = np.unique(df["Batch"])

In [214]:
metric ="Cumulative returns"
fig_title = f"{metric} per trading window | {setting}"

ncols = 4
nrows = generate_nrows(len(batch_names), ncols)

titles = generate_subplot_titles(batch_names)

fig = init_subplots(nrows, ncols, titles)
fig = populate_subplots(fig, df, batch_names, titles, ncols, y_varname=metric, initial_invest=initial_invest)

layout = generate_layout(
    fig_height=1200, 
    fig_width=1400, 
    fig_title=fig_title)
    
fig.update_layout(layout)

In [215]:
df_barplot = generate_df_barplot(df)
title = f"% Cumulative returns at the end of each trading window | {setting}"

layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title=title,
    x_axis_args={"title": {"text": "Trading window"}}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "%"}}
)
fig = make_cumrets_barplot(df_barplot, layout)

fig.show()

#### Portfolio allocation

In [216]:
positions = results[setting]["positions"]

avg_positions_per_window = np.array([p.mean(axis=1) for p in positions]) 
avg_positions = avg_positions_per_window.mean(axis=0)

df_pie = pd.DataFrame({
    "Asset": results[setting]["assets"], 
    "% Share": 100 * avg_positions
})

In [217]:
import plotly.express as px

palette = px.colors.sequential.Blues
palette.reverse

fig = px.pie(
    df_pie, 
    values="% Share", 
    names="Asset", 
    hole=.4, 
    color_discrete_sequence=palette)

layout = generate_layout(
    fig_height=500, 
    fig_width=700, 
    fig_title="Average portfolio allocation over trading windows"
)
fig.update_layout(layout)

fig.show()